In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
mofid = pd.read_csv("../data/main_dataset_mofid_data_V2.csv", index_col=0)
mofid = mofid[["smiles_linkers", "smiles_nodes", "cifname"]].set_index("cifname")
mordred = pd.read_csv("../data/all_f_main_dataset_mordred_V2.csv", index_col=0)
zeo = pd.read_csv("../data/main_dataset_zeo_V2.csv", index_col=0)
cif = pd.read_csv("../data/main_dataset_cif_property_V2.csv", index_col=0)



In [3]:
cif

,a,b,c,alpha,beta,gamma,volume,sg_number
initial MOF-5-C7 2040922.cif,12.852600,12.852600,12.852600,90.000000,90.000000,90.000000,2123.112341,221.0
to MOF-5(C7)_desolvated phase 2040923.cif,12.786337,12.786337,12.786337,84.507765,84.507765,84.507765,2063.376733,148.0
from Cu(I)-MOF-1 2084812.cif,13.393384,13.393384,13.393384,106.730376,106.730376,115.104051,1835.938608,88.0
from Cu(I)Cu(II)-MOF-2 2084857.cif,13.124566,13.124566,13.124566,107.741062,107.741062,112.989882,1735.380592,88.0
to Cu(II)-MOF-3 1524653.cif,10.044000,13.671000,13.671000,90.000000,90.000000,90.000000,2081.642704,60.0
...,...,...,...,...,...,...,...,...
1b_40 bar(R).cif,8.249900,18.659300,18.659300,90.000000,90.000000,90.000000,2872.363364,90.0
from 1a.cif,8.047300,18.855000,18.855000,90.000000,90.000000,90.000000,2860.903871,90.0
to 1b.cif,7.797900,16.850000,16.850000,90.000000,90.000000,90.000000,2214.656236,18.0
as-synthesized SNU-80a (initial) 842625.cif,12.223300,24.807600,24.807600,90.000000,90.000000,90.000000,7522.426833,134.0


In [4]:
mordred.shape, zeo.shape, cif.shape

((151, 1826), (162, 3), (165, 8))

In [5]:
def make_mordred():
    from mordred import Calculator, descriptors
    from rdkit import Chem
    mofid = pd.read_csv("../data/main_dataset_mofid_data_V2.csv", index_col=0)
    mofid = mofid[["smiles_linkers", "smiles_nodes", "cifname"]].set_index("cifname")
    linkers = [mofid.smiles_linkers[i].replace("'", "")[1:-1].split(", ") for i in range(mofid.__len__())]
    mols = [[Chem.MolFromSmiles(smi) for smi in smi_list] for smi_list in linkers]
    calc = Calculator(descriptors, ignore_3D=False)
    def f(mof):
        try: return calc.pandas(mof)
        except TypeError:
            return None
    dfs = [f(mof) for mof in mols]
    data_mordred = pd.DataFrame(columns=dfs[0].columns)

    for i, filename in enumerate(mofid.index):
        try:
            if linkers[i] != [""]:
                data_mordred.loc[filename] = dfs[i].mean()
        except AttributeError:
            print(f"{filename:_^20}")
    data_mordred = data_mordred.set_index(data_mordred.index.map(lambda name: name.replace(" ", "")))
    #data_mordred.to_csv("../data/all_f_main_dataset_mordred_V2.csv")
    return data_mordred



In [6]:
def metal_from_node(node: str):
    import re
    """
    input: smilesNodes: str
    return: 
    'metals' in node: list
    'unique' types of metals: np.array, dtype='<U2'
    'count' of unique: np.array, dtype=int
    """
    # "O[Zr]123(O)[OH]4[Zr]56([O]3[Zr]37([OH]2[Zr]28([O]1[Zr]14([O]6[Zr]([OH]53)([OH]21)([O]78)(O)O)([OH2])([OH2])O)[OH2])([OH2])([OH2])O)[OH2]"
    node = node.replace("OH", "").replace("O", "")
    node = node.replace("[", "").replace("]", "").replace(")", "").replace("(", "").replace(",", "")
    node = re.sub(r"\d", "", node) # replace numbers
    #print(node)
    # "ZrZrZrZrZrZr"
    start_cut = 0
    metals = []
    for i, char in enumerate(node[1:]):
        if not char.islower():
            metals.append(node[start_cut:i+1])
            start_cut = i+1
    metals.append(node[start_cut:])
    unique, counts = np.unique(np.array(metals), return_counts=True)
    return metals, unique, counts

In [7]:
elemental_descriptors = pd.read_csv("../data/elemental_descriptors.csv")

In [8]:
node_descriptors = pd.DataFrame(columns=("n_metals", 'n_types_metals', 'Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)'))

d = ['Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)']

for filename, node in zip(mofid.index.map(lambda name: name.replace(" ", "").replace(".cif", "")), mofid["smiles_nodes"]):
    try:
       metals, unique, count = metal_from_node(node.replace("'", "").replace(" ", "").replace("%", ""))
       #print(metals)
       n_metals = count.sum()
       n_metals_types = count.__len__()
       df = pd.DataFrame(columns=d, index=[range(count.sum())])
       for metal in metals:
          #print(metal)
          df.loc[len(df)] = elemental_descriptors.loc[elemental_descriptors["Symbol"] == metal].loc[:,d].iloc[0]
       node_descriptors.loc[filename] = n_metals, n_metals_types, *df.mean().array
    except IndexError:
       print(f"Error with {node}")

Error with []
Error with []
Error with []
Error with []
Error with ['*']
Error with ['*']
Error with []
Error with []
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']


In [9]:
node_descriptors = node_descriptors.loc[node_descriptors["n_types_metals"] == 1.]


In [10]:
zeo.index = zeo.index.map(lambda s: s.replace(".res", ""))

In [11]:
data_pld_lcd = zeo[["num1", "num2"]].rename({"num1": "lcd", "num2": "pld"}, axis=1)

In [12]:
cif = cif.rename({ "sg_number": "spacegroupNumber"}, axis=1)

In [13]:
cif.index = cif.index.map(lambda name: name.replace(" ", "").replace(".cif", ""))

In [14]:
cif = cif.set_index(cif.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [15]:
node_descriptors = node_descriptors.set_index(node_descriptors.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [16]:
data_pld_lcd = data_pld_lcd.set_index(data_pld_lcd.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [17]:
mordred = mordred.set_index(mordred.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [18]:
data_pld_lcd.shape, cif.shape, mordred.shape, node_descriptors.shape

((162, 2), (165, 8), (151, 1826), (145, 8))

In [19]:
data_main = pd.concat([data_pld_lcd, cif, mordred, node_descriptors], axis=1)

In [20]:
data_main = data_main.loc[list({*list(data_pld_lcd.index)} & {*list(
    cif.index)} & {*list(mordred.index)} & {*list(node_descriptors.index)})]


In [21]:
data_main.shape

(138, 1844)

In [22]:
# data_main.to_csv("../data/main_no_marked.csv")

In [23]:
db_from_to = pd.read_csv("../data/DB_main_with_SG_for_EDA.csv", sep=';')

In [24]:
db_from_to.CIF_init = db_from_to.CIF_init.map(lambda name: name.replace("(", "").replace(")", "").replace(" ", "").replace(".cif", ""))
db_from_to.CIF_final = db_from_to.CIF_final.map(lambda name: name.replace("(", "").replace(")", "").replace(" ", "").replace(".cif", ""))


In [25]:
marked = pd.DataFrame(columns=["target"])

for ind in db_from_to.index:
    if db_from_to.loc[ind, "Stimuli"].find("solvent") != -1 or db_from_to.loc[ind, "Stimuli"].find("gas") != -1 or db_from_to.loc[ind, "Stimuli"].find("humidity") != -1:
        marked.loc[db_from_to.loc[ind, "CIF_init"]] = [1]
        if db_from_to.loc[ind, "Reversible"] == "yes":
            marked.loc[db_from_to.loc[ind, "CIF_final"]] = [1]
        else:
            marked.loc[db_from_to.loc[ind, "CIF_final"]] = [0]
    else:
        marked.loc[db_from_to.loc[ind, "CIF_init"]] = [0]
        marked.loc[db_from_to.loc[ind, "CIF_final"]] = [0]

In [26]:
from pteproc_model import PreprocessingModel
model = joblib.load("../qmof_datasets/scaler.pkl")

In [27]:
data_main.shape

(138, 1844)

In [28]:
main_dataset = data_main[model.cols]

In [29]:
marked_dataset = pd.concat([data_main, marked], axis=1)

In [30]:
marked_dataset = marked_dataset.loc[list({*list(main_dataset.index)} & {*list(marked.index)})]

In [31]:
marked_dataset.shape

(102, 1845)

In [32]:
target, counts = np.unique(marked_dataset.target, return_counts=True)


print(f"|{'target':^10}|{'counts':^10}|")
print(f"{'-':-^23}")
print(f"|{'yes':^10}|{counts[0]:^10}|")
print(f"|{'no':^10}|{counts[1]:^10}|")
print(f"total count:{marked_dataset.__len__()}")

|  target  |  counts  |
-----------------------
|   yes    |    62    |
|    no    |    40    |
total count:102


In [33]:
marked_dataset.shape

(102, 1845)

In [34]:
x, y = model.transform(marked_dataset.drop(['target'], axis=1)), marked_dataset['target']

In [35]:
x.shape, x.dropna().shape

((102, 1018), (102, 1018))

In [39]:
for i in x.index:
    print(i)

toCAU13_-1GPa
fromassynthesizedDUT-30Zn804770
fromDMFsolvatedFebdp100K1058447
toVMOP-+1590348
fromic035111o_si_001
toCuII-MOF-31524653
fromja101541s_si_004
toDUT-49Zn100K2014973
toSTA-26Zr-Cchangedsymmetry1571656
fromMOF2fa1052168
to1lpMOF1944821
fromSION-21503701
to1b
toDUT-49Mn100K2014971
fromDUT-49Zn296K890363
fromHIMS-741884101
initialSTA-26Zr1571655
from1041925
2from91045135
fromUTSA-741884102
toDUT-49Co100K2014974
toMOF-74Zn1512654
fromCo-MOF1912199
toja500530y_si_003
toDUT-49Ni100K2014970
from1in5636900
toSION-11503702
toFdd21021920
toanie202202073-sup-0001-nku-128_toluene
toic035111o_si_003
fromUPC-6001942055
fromja500530y_si_003
BasedonMntoR-3c
toCCDC_2065039_JUK-20-chx
from3in61515548
from1withDMF1403842
to922086
tocompoundwithmethanol1403845
from975784
fromDUT-49Mn296K1957604
basedonCofromPnna709782
to41483725
1from61055570
from11974527
toCo-MOF1912233
fromic035111o_si_002
2to101045134
to6cm6b05277_si_005-Copy
fromC2_c1021935
1initial924307
from1934734
from2D854460
INITIALan

In [42]:
x = x.drop("frompart_2", axis=0)

,pld,lcd,a,b,c,alpha,beta,gamma,volume,spacegroupNumber,...,Zagreb2,mZagreb1,mZagreb2,n_metals,Atomic_Number,Atomic_Weight,Atomic Radius,Mulliken EN,polarizability(A^3),electron affinity(kJ/mol)
toCAU13_-1GPa,0.162791,0.137948,0.101440,0.169292,0.077769,0.441237,0.274418,0.199779,0.010622,0.048458,...,0.081009,0.113257,0.077694,0.000000,0.112360,0.086723,0.000000,0.422680,0.063869,0.158865
fromassynthesizedDUT-30Zn804770,0.181673,0.170747,0.189917,0.285933,0.157241,0.510829,0.258064,0.186620,0.033658,0.299559,...,0.282869,0.218264,0.268170,0.090909,0.303371,0.252930,0.133333,0.841924,0.029197,0.014184
fromDMFsolvatedFebdp100K1058447,0.036981,0.041617,0.305017,0.282025,0.154579,0.578143,0.694983,0.441638,0.095942,0.057269,...,0.128818,0.059254,0.115288,0.000000,0.258427,0.211634,0.211111,0.707904,0.065693,0.120567
toVMOP-+1590348,0.152576,0.280814,0.552444,0.524591,0.319849,0.787016,0.810204,0.687872,0.154692,0.951542,...,0.040837,0.150734,0.027882,0.000000,0.224719,0.190406,0.294444,0.549828,0.138686,0.171631
fromic035111o_si_001,-0.007198,0.015769,0.361295,0.364577,0.210825,0.578143,0.526960,0.441638,0.112044,0.264317,...,0.104914,0.076505,0.110276,0.000000,0.269663,0.224989,0.188889,0.790378,0.049270,0.190071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
from4in61536823,0.177338,0.202033,0.399773,0.366884,0.212397,0.578329,0.527212,0.062898,0.123227,0.017621,...,0.100266,0.109882,0.103383,0.000000,0.280899,0.223951,0.172222,0.824742,0.036496,0.258156
2from2924308-Copy,0.106497,0.110516,0.122000,0.296370,0.164353,0.725332,0.560694,0.483130,0.026897,0.004405,...,0.146082,0.099006,0.122807,0.000000,0.303371,0.252930,0.133333,0.841924,0.029197,0.014184
fromDUT-49Cd296K1957602,0.439085,0.587208,0.914024,0.898095,0.574334,0.256326,0.090555,0.062258,0.540158,0.986784,...,0.164675,0.145509,0.137845,0.090909,0.505618,0.456406,0.238889,0.800687,0.043796,0.000000
basedonMnfromPnna709780,0.039804,0.023974,0.156166,0.132090,0.052422,0.578143,0.526960,0.441638,0.015372,0.224670,...,0.003984,0.027002,0.000000,0.000000,0.247191,0.207700,0.238889,0.591065,0.083942,0.028369


In [37]:
x.to_csv("../main_datasets/dataset.csv")

In [38]:
y.to_csv("../main_datasets/target.csv")